In [173]:
import pandas as pd
import numpy as np

In [174]:
df = pd.read_csv('/content/wine.csv')

In [175]:
#sorting according to sum scores
data = df.to_numpy()
def avg(point):
  list1 = []
  for i in range(point.shape[0]):
    list1.append(point[i])
  return sum(list1)
averages = np.apply_along_axis(avg, 1, data)
sorted_indices = np.argsort(averages)
modified_data = data[sorted_indices]

In [176]:
#splitting the dataset into k parts
k = 3
split_indices = np.linspace(0, modified_data.shape[0], num=k+1, dtype=int)
parts = np.split(modified_data, split_indices[1:-1])

In [177]:
#calculating initial centroids
cluster_centers = []
for i, part in enumerate(parts):
    size = part.shape[1]
    arr = []
    for j in range(size):
      x = np.median(part[: , j])
      arr.append(x)
    cluster_centers.append(arr)
cluster_centers = np.array(cluster_centers)

In [178]:
centroids = pd.DataFrame(cluster_centers)
centroids['cluster'] = range(k)

In [179]:
new_df = pd.DataFrame(modified_data[:, :])
new_df['cluster'] = 0
c_arr=np.array(centroids)
dt_arr=np.array(new_df)

In [180]:
for iteration in range (100):
    for i in range(dt_arr.shape[0]):
        dist=[]
        for c in c_arr:
            d=np.linalg.norm(dt_arr[i,:-1]-c[:-1])
            dist.append(d)
        clus=dist.index(np.min(dist))
        dt_arr[i,-1]=clus

    for c in range(k):
        c_arr[c,:-1]=np.mean(dt_arr[dt_arr[:,-1]==c][:,:-1],axis=0)

In [181]:
cluster_indices = dt_arr[ :,-1]

In [182]:
from sklearn.metrics import silhouette_score
from sklearn import metrics
from sklearn.metrics import pairwise_distances

In [183]:
print(silhouette_score(dt_arr[:,:-1],dt_arr[:,-1]))
print(metrics.calinski_harabasz_score(dt_arr[:,:-1],dt_arr[:,-1]))
print(metrics.davies_bouldin_score(dt_arr[:,:-1],dt_arr[:,-1]))

0.5709784028530243
559.6932245530587
0.5336393583771107


In [184]:
#sum squared error
sse = 0
for i in range(k):
  cluster_points = dt_arr[dt_arr[:,-1] == i][:,:-1]
  cluster_center = c_arr[i]
  for j in range(cluster_points.shape[0]):
    cluster_point = cluster_points[j]
    x = np.linalg.norm(cluster_point - cluster_center[:-1])
    sse += x**2
print(sse)

2352907.766903644
